In [3]:
import os
import importlib.util
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# 最適化関数を取得する関数
def get_optimizer(optimizer_name, params, lr):
    if optimizer_name == 'SGD':
        return optim.SGD(params, lr)
    if optimizer_name == 'momentum':
        return optim.SGD(params, lr, momentum=0.8)
    if optimizer_name == 'Adam':
        return optim.Adam(params, lr)
    if optimizer_name == 'Adagrad':
        return optim.Adagrad(params, lr)
    if optimizer_name == 'RMSProp':
        return optim.RMSprop(params, lr)
    if optimizer_name == 'Adadelta':
        return optim.Adadelta(params, lr)

# 損失関数を取得する関数
def get_loss(loss_name):
    if loss_name == 'mse_loss':
        return nn.MSELoss()
    if loss_name == 'cross_entropy':
        return nn.CrossEntropyLoss()
    if loss_name == 'binary_cross_entropy':
        return nn.BCELoss()
    if loss_name == 'nll_loss':
        return nn.NLLLoss()
    if loss_name == 'hinge_embedding_loss':
        return nn.HingeEmbeddingLoss()

def load_model(config):
    user_id = config["user_id"]
    project_name = config["Project_name"]
    model_id = config["model_id"]
    
    base_dir = "../user"
    model_path = os.path.join(base_dir, user_id, project_name, model_id, "model_config.py")
    
    spec = importlib.util.spec_from_file_location("Simple_NN", model_path)
    model_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(model_module)
    Simple_NN = model_module.Simple_NN
    
    return Simple_NN

def accuracy(outputs, targets):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == targets).sum().item()
    return correct / targets.size(0)

def train_model(config):
    user_id = config["user_id"]
    project_name = config["Project_name"]
    model_id = config["model_id"]
    train_info = config["Train_info"]

    base_dir = "../user"

    # モデルのロード
    Simple_NN = load_model(config)
    model = Simple_NN()

    # 学習情報の取得
    loss_function = get_loss(train_info["loss"])
    optimizer = get_optimizer(train_info["optimizer"], model.parameters(), train_info["learning_rate"])
    batch_size = train_info["batch"]
    epochs = train_info["epoch"]

    # 仮のデータセット (実際のデータを使用する場合はここを変更)
    x_train = torch.randn(100, 10)  # 入力データ
    y_train = torch.randint(0, 2, (100,))  # ターゲットデータ
    dataset = torch.utils.data.TensorDataset(x_train, y_train)
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # 学習ループ
    loss_values = []
    accuracy_values = []
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        correct = 0
        total = 0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, targets)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

            # 正答率の計算
            acc = accuracy(outputs, targets)
            accuracy_values.append(acc)
        
        avg_epoch_loss = epoch_loss / len(train_loader)
        loss_values.append(avg_epoch_loss)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_epoch_loss:.4f}, Accuracy: {acc:.4f}")

    # 学習曲線と正答率のプロット
    plt.figure()
    plt.plot(loss_values, label="Training Loss")
    plt.plot(accuracy_values, label="Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Value")
    plt.title("Training Loss and Accuracy Curve")
    plt.legend()
    plt.grid(True)

    # ディレクトリの作成と保存
    photo_dir = os.path.join(base_dir, user_id, project_name, model_id, "photo")
    os.makedirs(photo_dir, exist_ok=True)
    plt.savefig(os.path.join(photo_dir, "training_loss_accuracy_curve.png"))
    plt.close()

# 引数として渡すオブジェクト
config = {
    "user_id": "example_user_id",
    "Project_name": "example_project",
    "model_id": "example_model",
    "Train_info": {
        "Pretreatment": "none",
        "loss": "cross_entropy",
        "optimizer": "Adam",
        "learning_rate": 0.001,
        "batch": 32,
        "epoch": 10
    }
}

train_model(config)


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [32, 10]